# CPSC 368 Impact By Sex (KNM Neighbours)
## Import Data and Packages

In [1]:
import oracledb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

For the coronary heart disease mortality dataset, the U.S. Chronic Disease Indicators dataset is filtered for the corresponding cases, with the common unit being `USCDI["DataValueUnit"] == 'cases per 100,000'` and with the stratification categories `Sex` and `Age`. 
- Sex is used to estimate the proportion of each gender within each location. This is achieved by obtaining the sum of cases per 100,000 people for each location and gender, regardless of age, followed by calculating the proportion of female individuals present.
- Age is used to get the appropriate age group, with the closest achievable groups being the sum of cases per 100,000 people between `Age 0-44` and `Age 45-64`.
- Finally, the proportion of individuals that had coronary heart disease is calculated, along with the corresponding proportions for each gender, by dividing their values by 100000.
- The column `AvgDataValue` is renamed to `CHD_Deaths` to make future interpretation easier for users.

In [2]:
# INPUT USER INFO HERE
CWL = "nick2003"
studentnum = "45695970"

In [3]:
# Acquire data
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user=f"ora_{CWL}", password=f"a{studentnum}", dsn=dsn)
cur = connection.cursor()

try:
    # Remove USCDI_CHD view only if it exists
    cur.execute("SELECT COUNT(*) FROM ALL_VIEWS WHERE VIEW_NAME = 'USCDI_CHD'")
    view_exists = cur.fetchone()[0]
    if view_exists:
        cur.execute("DROP VIEW USCDI_CHD CASCADE CONSTRAINTS")
    
    # Create USCDI_CHD
    cur.execute(
        """
        CREATE VIEW USCDI_CHD AS
            WITH CHD_Data AS (
                SELECT 
                    total.LocationDesc AS LocationDesc,
                    CAST(female.AvgDataValue / (female.AvgDataValue + male.AvgDataValue) AS DECIMAL(19, 18)) AS Frac_F,
                    CAST(total.AvgDataValue AS DECIMAL(24, 18)) AS CHD_Deaths
                FROM 
                    (SELECT LocationDesc, AvgDataValue
                    FROM USCDI
                    WHERE Topic = 'Cardiovascular Diseases'
                    AND Question = 'Death rate from coronary heart disease (CHD)'
                    AND DataValueUnit = 'cases per 100,000'
                    AND StratificationCategory1 = 'Age'
                    AND Stratification1 IN ('Age 0-44', 'Age 45-64')
                    AND DataValueType = 'Crude Rate'
                    AND Has2019 = 1) total
                JOIN
                    (SELECT LocationDesc, AvgDataValue
                    FROM USCDI
                    WHERE Topic = 'Cardiovascular Diseases'
                    AND Question = 'Death rate from coronary heart disease (CHD)'
                    AND DataValueUnit = 'cases per 100,000'
                    AND StratificationCategory1 = 'Sex'
                    AND Stratification1 = 'Female'
                    AND DataValueType = 'Age-adjusted Rate'
                    AND Has2019 = 1) female
                ON total.LocationDesc = female.LocationDesc
                JOIN
                    (SELECT LocationDesc, AvgDataValue
                    FROM USCDI
                    WHERE Topic = 'Cardiovascular Diseases'
                    AND Question = 'Death rate from coronary heart disease (CHD)'
                    AND DataValueUnit = 'cases per 100,000'
                    AND StratificationCategory1 = 'Sex'
                    AND Stratification1 = 'Male'
                    AND DataValueType = 'Age-adjusted Rate'
                    AND Has2019 = 1) male
                ON total.LocationDesc = male.LocationDesc
            )
            SELECT 
                CHD_Data.LocationDesc,
                CHD_Data.Frac_F,
                CHD_Data.CHD_Deaths,
                CAST(CHD_Data.CHD_Deaths * CHD_Data.Frac_F AS DECIMAL(24, 18)) AS CHD_Deaths_F,
                CAST(CHD_Data.CHD_Deaths * (1 - CHD_Data.Frac_F) AS DECIMAL(24, 18)) AS CHD_Deaths_M,
                CAST(CHD_Data.CHD_Deaths / 1000 AS DECIMAL(19, 18)) AS CHDPercentage,
                CAST((CHD_Data.CHD_Deaths * CHD_Data.Frac_F) / 1000 AS DECIMAL(19, 18)) AS CHDPercentage_F,
                CAST((CHD_Data.CHD_Deaths * (1 - CHD_Data.Frac_F)) / 1000 AS DECIMAL(19, 18)) AS CHDPercentage_M
            FROM CHD_Data
        """
    )

    cur.execute(
        """
        SELECT
            uc.LocationDesc,
            uc.Frac_F,
            uc.CHD_Deaths,
            uc.CHD_Deaths_F,
            uc.CHD_Deaths_M,
            uc.CHDPercentage,
            uc.CHDPercentage_F,
            uc.CHDPercentage_M,
            kff.All_Uninsured,
            kff.Female_Uninsured,
            kff.Male_Uninsured
        FROM USCDI_CHD uc
        LEFT JOIN KFF2019_new kff
            ON uc.LocationDesc = kff.Location
        """
    )
    
    # Fetch SELECT results 
    rows = cur.fetchall()
    
    # Get column names from cursor
    columns = [desc[0] for desc in cur.description]
    
    # Create DataFrame
    total_data = pd.DataFrame(rows, columns=columns)
    
    print("Completed dataframe.")
except Exception as e:
    print(f"Error executing SQL query: {e}")
finally: 
    cur.close()
    connection.close()

Completed dataframe.


In [4]:
# USCDI_CHD = pd.read_csv("final_datasets_V1/cleaned/USCDI_CHD.csv")
# KFF2019_new = pd.read_csv("final_datasets_V1/cleaned/KFF2019_new.csv")
# total_data = pd.merge(USCDI_CHD, KFF2019_new, left_on='LOCATIONDESC', right_on='Location', how='left')

## EDA
For this analysis, the focus will be on uninsured rates and coronary heart disease (CHD) mortality rates across males and females in Texas and Massachusetts. 

`facet_1` displays bars for `CHDPERCENTAGE` by location and sex, with the `CHDPERCENTAGE_M` values being  greater than the corresponding `CHDPERCENTAGE_F` value for both states. This supports existing research that indicates that CHD incidence and mortality rates have historically been higher in men than women between the ages 35 and 84, though the difference in morbidity between sexes decreases with age (Lerner, Kannel, 1986).

In [5]:
total_data_focus = total_data[(total_data['LOCATIONDESC'].isin(['Texas', 'Massachusetts']))]

In [6]:
facet_1 = alt.Chart(total_data_focus).mark_bar().encode(
    alt.Y("LOCATIONDESC:N"),
    alt.X(alt.repeat('row'), 
          type='quantitative', 
          scale=alt.Scale(domain=[0,max(total_data_focus['CHDPERCENTAGE_F'].max(), total_data_focus['CHDPERCENTAGE_M'].max())]))
).repeat(
  row=['CHDPERCENTAGE_F', 'CHDPERCENTAGE_M',]
).properties(
    title="CHD Percentage for Females and Males by Location"
)
facet_1

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.RepeatChart(...)

`facet_2` displays bars for uninsurance rate by location and sex, with the `MALE_UNINSURED` values being  greater than the corresponding `FEMALE_UNINSURED` value for both states.

In [7]:
facet_2 = alt.Chart(total_data_focus).mark_bar().encode(
    alt.Y("LOCATIONDESC:N"),
    alt.X(alt.repeat('row'), 
          type='quantitative', 
          scale=alt.Scale(domain=[0,max(total_data_focus['FEMALE_UNINSURED'].max(), total_data_focus['MALE_UNINSURED'].max())]))
).repeat(
  row=['FEMALE_UNINSURED', 'MALE_UNINSURED',]
).properties(
    title="Percentage of Uninsured Individuals for Females and Males by Location"
)
facet_2

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.RepeatChart(...)

`plot_3` displays bars for the ratio of the percentage of uninsured individuals over the percentage of coronary heart disease (CHD) mortality rates by location and sex, with the `CHD_Uninsured_Ratio_F` values being lower than the corresponding `CHD_Uninsured_Ratio_M` value for both states. This implies that uninsured females are at a relatively lower risk of CHD mortality than uninsured males.

In [8]:
total_data_focus["CHD_Uninsured_Ratio_F"] = total_data_focus['CHDPERCENTAGE_F'] / total_data_focus['FEMALE_UNINSURED']
total_data_focus["CHD_Uninsured_Ratio_M"] = total_data_focus['CHDPERCENTAGE_M'] / total_data_focus['MALE_UNINSURED']

plot_3 = alt.Chart(total_data_focus).mark_bar().encode(
    alt.Y("LOCATIONDESC:N"),
    alt.X(alt.repeat('row'), 
          type='quantitative', 
          scale=alt.Scale(domain=[0,max(total_data_focus['CHD_Uninsured_Ratio_F'].max(), total_data_focus['CHD_Uninsured_Ratio_M'].max())]))
).repeat(
  row=['CHD_Uninsured_Ratio_F', 'CHD_Uninsured_Ratio_M',]
).properties(
    title="Ratio of CHD Mortality Percentage over Uninsured Percentage for Females and Males by Location"
)
plot_3

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.RepeatChart(...)

## References
- Lerner, D. J., & Kannel, W. B. (1986). Patterns of coronary heart disease morbidity and mortality in the sexes: a 26-year follow-up of the Framingham population. American heart journal, 111(2), 383–390. https://doi.org/10.1016/0002-8703(86)90155-9